In [1]:
import os, shutil, random, glob, cv2
import numpy as np
from tqdm.autonotebook import tqdm

C:\Users\fhacesga\AppData\Local\Temp\ipykernel_32072\3312261161.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
def old_tile_image(image, boxes, tile_size=512, overlap=0.2):
    """ 
    Tile the image and adjust bounding boxes accordingly.
    Args:
        image: The image as a numpy array.
        boxes: List of bounding boxes in YOLO format [class_id, x_center, y_center, width, height].
        tile_size: Size of each tile (default 512x512).
        overlap: Fractional overlap between tiles (default 0.2 or 20% overlap).
    Returns:
        List of tiles with corresponding adjusted bounding boxes.
    """
    img_h, img_w = image.shape[:2]
    step_size = int(tile_size * (1 - overlap))  # Step size based on overlap
    tiles = []
    
    # Loop over the image, creating tiles with overlap
    for y in range(0, img_h, step_size):
        for x in range(0, img_w, step_size):
            # Extract tile region
            x_end = min(x + tile_size, img_w)
            y_end = min(y + tile_size, img_h)
            tile = image[y:y_end, x:x_end]
            
            # Adjust boxes for this tile
            adjusted_boxes = []
            for box in boxes:
                class_id, x_center_rel, y_center_rel, box_w_rel, box_h_rel = box
                # Convert relative box to absolute
                box_x_center = x_center_rel * img_w
                box_y_center = y_center_rel * img_h
                box_w = box_w_rel * img_w
                box_h = box_h_rel * img_h
                
                # Check if the bounding box is within this tile
                x_min = box_x_center - box_w / 2
                y_min = box_y_center - box_h / 2
                x_max = box_x_center + box_w / 2
                y_max = box_y_center + box_h / 2
                
                # If the bounding box is inside this tile, adjust it
                if x_min < x_end and x_max > x and y_min < y_end and y_max > y:
                    # Adjust coordinates relative to the tile
                    new_x_min = max(x_min, x) - x
                    new_y_min = max(y_min, y) - y
                    new_x_max = min(x_max, x_end) - x
                    new_y_max = min(y_max, y_end) - y
                    
                    # Scale them back to relative coordinates
                    new_x_center = (new_x_min + new_x_max) / 2 / tile_size
                    new_y_center = (new_y_min + new_y_max) / 2 / tile_size
                    new_box_w = (new_x_max - new_x_min) / tile_size
                    new_box_h = (new_y_max - new_y_min) / tile_size
                    
                    adjusted_boxes.append([class_id, new_x_center, new_y_center, new_box_w, new_box_h])
            
            # Save the tile and adjusted boxes if there are any boxes in this tile
            if len(adjusted_boxes) > 0:
                tiles.append((tile, adjusted_boxes))
    
    return tiles

def tile_image(image, boxes, tile_size=512, overlap=0.2):
    """ 
    Tile the image and adjust bounding boxes accordingly, padding the last tiles if needed.
    Args:
        image: The image as a numpy array.
        boxes: List of bounding boxes in YOLO format [class_id, x_center, y_center, width, height].
        tile_size: Size of each tile (default 512x512).
        overlap: Fractional overlap between tiles (default 0.2 or 20% overlap).
    Returns:
        List of tiles with corresponding adjusted bounding boxes.
    """
    img_h, img_w = image.shape[:2]
    step_size = int(tile_size * (1 - overlap))  # Step size based on overlap
    tiles = []

    # Calculate the number of tiles needed for width and height
    num_tiles_x = (img_w + step_size - 1) // step_size  # Total tiles in the x-direction
    num_tiles_y = (img_h + step_size - 1) // step_size  # Total tiles in the y-direction

    # Loop over the image based on the number of tiles needed
    for i in range(num_tiles_y):
        for j in range(num_tiles_x):

            

            # Calculate the region for the tile
            x = j * step_size
            y = i * step_size
            x_end = min(x + tile_size, img_w)
            y_end = min(y + tile_size, img_h)

            #print(i, j, img_w, img_h, x, y, x_end, y_end)
            
            # Pad the tile if necessary (for last tiles in the row/column)
            tile = np.full((tile_size, tile_size, 3), 255, dtype=np.uint8)  # Create a white (255) tile
            tile[0:(y_end - y), 0:(x_end - x)] = image[y:y_end, x:x_end]  # Place the image portion in the tile
            
            # Adjust boxes for this tile
            adjusted_boxes = []
            for box in boxes:
                class_id, x_center_rel, y_center_rel, box_w_rel, box_h_rel = box
                # Convert relative box to absolute
                box_x_center = x_center_rel * img_w
                box_y_center = y_center_rel * img_h
                box_w = box_w_rel * img_w
                box_h = box_h_rel * img_h

                # Check if the bounding box is within this tile
                x_min = box_x_center - box_w / 2
                y_min = box_y_center - box_h / 2
                x_max = box_x_center + box_w / 2
                y_max = box_y_center + box_h / 2

                # If the bounding box is inside this tile, adjust it
                if x_min < x_end and x_max > x and y_min < y_end and y_max > y:
                    # Adjust coordinates relative to the tile
                    new_x_min = max(x_min, x) - x
                    new_y_min = max(y_min, y) - y
                    new_x_max = min(x_max, x_end) - x
                    new_y_max = min(y_max, y_end) - y

                    # Scale them back to relative coordinates
                    new_x_center = (new_x_min + new_x_max) / 2 / tile_size
                    new_y_center = (new_y_min + new_y_max) / 2 / tile_size
                    new_box_w = (new_x_max - new_x_min) / tile_size
                    new_box_h = (new_y_max - new_y_min) / tile_size

                    adjusted_boxes.append([class_id, new_x_center, new_y_center, new_box_w, new_box_h])

            # Save the tile and adjusted boxes if there are any boxes in this tile
            tiles.append((tile, adjusted_boxes))

    return tiles

def save_tiles(tiles, output_dir, img_id, label_id):
    """ Save the tiles and their corresponding annotations. """
    os.makedirs(output_dir, exist_ok=True)

    image_dir = os.path.join(output_dir, "images/all/")
    label_dir = os.path.join(output_dir, "labels/all/")
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for i, (tile, boxes) in enumerate(tiles):
        # Save tile image
        tile_img_path = os.path.join(image_dir, f'{img_id}_tile_{i}.jpg')
        cv2.imwrite(tile_img_path, tile)

        # Save corresponding label file
        label_file_path = os.path.join(label_dir, f'{label_id}_tile_{i}.txt')
        with open(label_file_path, 'w') as f:
            for box in boxes:
                class_id, x_center, y_center, width, height = box
                f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

def process_dataset(image_dir, label_dir, output_dir, tile_size=512, overlap=0.2):
    """
    Process the dataset by tiling the images and adjusting bounding boxes.
    Args:
        image_dir: Path to the directory containing images.
        label_dir: Path to the directory containing YOLO labels.
        output_dir: Path to the directory where output tiles and labels will be saved.
        tile_size: Size of each tile.
        overlap: Fractional overlap between tiles.
    """
    for img_file in tqdm(os.listdir(image_dir)):
        if img_file.endswith(('.jpg', '.png', '.tif')):
            # Load image and corresponding label
            img_id = os.path.splitext(img_file)[0]
            img_path = os.path.join(image_dir, img_file)
            label_path = os.path.join(label_dir, f"{img_id}.txt")
            
            image = cv2.imread(img_path)
            boxes = []
            
            # Read labels
            if os.path.exists(label_path):
                with open(label_path, 'r') as f:
                    for line in f:
                        box = list(map(float, line.strip().split()))
                        boxes.append(box)
            
            # Tile the image and adjust bounding boxes
            tiles = tile_image(image, boxes, tile_size=tile_size, overlap=overlap)
            
            # Save tiles and labels
            save_tiles(tiles, output_dir, img_id, img_id)

def copy_matching_images(text_dir, image_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Get list of filenames in directory A
    text_files = [os.path.splitext(file)[0] for file in os.listdir(text_dir)]
    
    # Get list of filenames in directory B
    image_files = [os.path.splitext(file)[0] for file in os.listdir(image_dir)]
    
    # Find matching filenames
    matching_files = set(text_files) & set(image_files)
    # Copy matching image files to directory C
    for filename in matching_files:
        fn = glob.glob(os.path.join(image_dir, filename)+"*")
        shutil.copy(fn[0], output_dir)

def split_train_val(yolo_dir, percentage_train=80):
    os.makedirs(yolo_dir + "labels/train/", exist_ok=True)
    os.makedirs(yolo_dir + "labels/val/", exist_ok=True)
    # Get list of filenames in directory A
    text_files = os.listdir(yolo_dir + "labels/all/")

    # Calculate the number of filenames to select
    num_files_to_select = int(len(text_files) * percentage_train / 100)
    
    # Randomly select filenames
    selected_files = random.sample(text_files, num_files_to_select)
    print(selected_files)
    for file in selected_files: 
        shutil.copy(os.path.join(yolo_dir + "labels/all/", file), yolo_dir + "labels/train/")

    not_selected_files = set(text_files) - set(selected_files)

    for file in not_selected_files: 
        shutil.copy(os.path.join(yolo_dir + "labels/all/", file), yolo_dir + "labels/val/")

    copy_matching_images(yolo_dir + "labels/train/", yolo_dir + "images/all/", yolo_dir + "images/train/")
    copy_matching_images(yolo_dir + "labels/val/", yolo_dir + "images/all/", yolo_dir + "images/val/")



In [4]:
# Example usage
image_dir = r'D:\RECTDNN\YOLO\datasets\TPNNv4\images\all\\'
label_dir = r'D:\RECTDNN\YOLO\datasets\TPNNv4\labels\all\\'
output_dir = 'D:\RECTDNN\YOLO\datasets\TPNNv5\\'


process_dataset(image_dir, label_dir, output_dir, tile_size=640, overlap=0.2)
split_train_val(output_dir)

  0%|          | 0/12 [00:00<?, ?it/s]

['48201C0170G_tile_296.txt', '48201C0170G_tile_301.txt', '48201C0160G_tile_395.txt', '48201C0160G_tile_574.txt', '48201C0160G_tile_242.txt', '48201C0865J_tile_58.txt', '48201C0170G_tile_321.txt', '48201C0865J_tile_229.txt', '48201C0415L_tile_263.txt', '48201C0300G_tile_472.txt', '48201C0300G_tile_29.txt', '485469B_6_tile_16.txt', '48201C0300G_tile_479.txt', '48201C0140G_tile_459.txt', '48201C0170G_tile_91.txt', '48201C0865J_tile_324.txt', '48201C0415L_tile_423.txt', '48201C0865J_tile_409.txt', '485469B_6_tile_44.txt', '48201C0295L_tile_53.txt', '48201C0300G_tile_530.txt', '48201C0160G_tile_114.txt', '48201C0300G_tile_624.txt', '48201C0300G_tile_481.txt', '48201C0295L_tile_369.txt', '48201C0300G_tile_412.txt', '48201C0300G_tile_372.txt', '48201C0415L_tile_318.txt', '48201C0415L_tile_397.txt', '48201C0865J_tile_279.txt', '48201C0415L_tile_321.txt', '48201C0295L_tile_457.txt', '48201C0865J_tile_139.txt', '48201C0160G_tile_346.txt', '48201C0415L_tile_130.txt', '48201C0295L_tile_276.txt', '